# ДЗ №3 
## Обучение моделей глубокого обучения на PyTorch

In [123]:
import torch
import torchvision
import numpy as np

from typing import Tuple, List, Type, Dict, Any
from tqdm.notebook import tqdm as tqdm

In [124]:
torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#### Задание 1

Повторите реализацию трёхслойного перцептрона из предыдущего задания на **Pytorch**. Желательно также, чтобы реализация модели имела параметризуемую глубину ( количество слоёв ), количество параметров на каждом слое и функцию активации. Отсутствие такой возможности не снижает балл, но сильно поможет в освоении принципов построения нейросетей с применением библиотеки pytorch.

In [125]:
class Perceptron(torch.nn.Module):
    
    def __init__(self, 
                 input_resolution: Tuple[int, int] = (28, 28),
                 input_channels: int = 1, 
                 hidden_layer_features: List[int] = [256, 256, 256],
                 activation: Type[torch.nn.Module] = torch.nn.ReLU,
                 num_classes: int = 10):

        super().__init__()
        
        paramsOnLayer = [input_resolution[0] * input_resolution[1]]
        paramsOnLayer.extend(hidden_layer_features)
        paramsOnLayer.append(num_classes)
        
        layers = []
        for i in range(len(paramsOnLayer)-1):
            layers.append(torch.nn.Linear(in_features=paramsOnLayer[i], out_features=paramsOnLayer[i+1], bias=True))
            layers.append(activation())
            
        self.structure = torch.nn.Sequential(*layers)    
        
    
    def forward(self, x):
        x = self.structure(x)
        return x

### Задание 2

Обоснуйте, почему аугментация обучающей выборки позволяет добиться прироста качества модели, несмотря на то, что она не добавляет в неё дополнительную информацию.

#### Ответ
Уникальной дополнительной информации аугментация не даст. Однако она позволит модели обучиться отлавливать какие-либо изменения в данных. Если мы будем говорить в рамках предложенной статьи, то такими изменениями могут стать: повороты, сдвиги, изменения цвета, блюр и тд. <p>
Чем это полезно? Не переобучится ли наша модель? <p>
Если наша модель обучится находить на детализированных картинках различных котов, то на картинках с низким разрешением или просто на тех же, но размытых, она вряд ли сможет что-то найти. Поэтому мы и дополняем нашу обучающую выборку изображениями, которые будут слегка изменены.

### Задание 3

Какие осмысленные аугментации вы можете придумать для следующих наборов данных:

1. Набор изображений животных, размеченый на виды животных
2. Набор аудиозаписей голоса, размечеными на языки говорящего
3. Набор cо показаниями датчиков температуры, влажности и давления с одной из метеостанций, размеченый на признак наличия осадков

1. Размытие, поворот, небольшие растяжения и сжатия изображения, переход в greyscale  
2. Добовление шумов на фон (различные звуки), замедление или ускорение записей; понижение, повышение тона
3. Этот набор данных мне показался самым сложным. Как качественно добавить данных я не придумал, но есть пару мыслей. Можно в зависимости от температуры, пошуметь влажностью и давлением (при -20 снег при любых (почти) давлениях и влажностях остается снегом)

In [136]:
train_transforms = torchvision.transforms.Compose([    
    # решил сильно не преобразовывать данные
    # хотел ещё блюрить или наоборот резкость увеличивать,
    # но потом понял, что для такого набора данных это
    # будет излишнем
    torchvision.transforms.RandomRotation(20),   
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0, ), (0.3, ))
])

val_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0, ), (0.3, ))
])

In [127]:
train_dataset = torchvision.datasets.MNIST(root='./mnist', 
                                           train=True, 
                                           download=True,
                                           transform=train_transforms)

val_dataset = torchvision.datasets.MNIST(root='./mnist', 
                                         train=False, 
                                         download=True,
                                         transform=val_transforms)

In [128]:
def train_model(model: torch.nn.Module, 
                train_dataset: torch.utils.data.Dataset,
                val_dataset: torch.utils.data.Dataset,
                loss_function: torch.nn.Module = torch.nn.CrossEntropyLoss(),
                optimizer_class: Type[torch.optim.Optimizer] = torch.optim,
                optimizer_params: Dict = {},
                initial_lr = 0.01,
                lr_scheduler_class: Any = torch.optim.lr_scheduler.ReduceLROnPlateau,
                lr_scheduler_params: Dict = {},
                batch_size = 64,
                max_epochs = 1000,
                early_stopping_patience = 10):
    optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr, **optimizer_params)
    lr_scheduler = lr_scheduler_class(optimizer, **lr_scheduler_params)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

    best_val_loss = None
    best_epoch = None

    for epoch in range(max_epochs):
        
        print(f'------- Current epoch: {epoch} -------')
        
        train_single_epoch(model, optimizer, loss_function, train_loader)
        val_metrics = validate_single_epoch(model, loss_function, val_loader)
        
        print(f'Validation metrics: \n{val_metrics}')
        
        lr_scheduler.step(val_metrics['loss'])
        
        if best_val_loss is None or best_val_loss > val_metrics['loss']:
            print(f'Best model yet, saving')
            best_val_loss = val_metrics['loss']
            best_epoch = epoch
            torch.save(model, './best_model.pth')
            
        if epoch - best_epoch > early_stopping_patience:
            print('Early stopping triggered')
            return
        print()
            

In [129]:
def train_single_epoch(model: torch.nn.Module,
                       optimizer: torch.optim.Optimizer, 
                       loss_function: torch.nn.Module, 
                       data_loader: torch.utils.data.DataLoader):
    loss = None
    """
    У меня была идея связанная с тем, как сворачивать матрицу картинки в массив.
    https://www.youtube.com/watch?v=3s7h2MHQtxc
    Например, как показано в этом видеоролике, с помощью space filling curves, 
    мы смогли бы сохранить пространственное свойство картинок. 

    Однако, т.к. мы работаем с картинками одного и того же разрешения, нам это 
    никак не поможет. Но всё-таки идея, как мне кажется неплохая, поэтому я хотел бы узнать,
    что вы думаете по этому поводу. Как сильно я ошибаюсь и в чем не прав.
    """
    with tqdm(total=len(data_loader)) as pbar:
        pbar.set_description('training ')
        for x_batch, y_batch in data_loader:
            x_batch = x_batch.view(x_batch.shape[0], -1)

            y_pred = model(x_batch)
            loss = loss_function(y_pred, y_batch)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pbar.update(1)


In [130]:
def validate_single_epoch(model: torch.nn.Module,
                          loss_function: torch.nn.Module, 
                          data_loader: torch.utils.data.DataLoader):
    loss_history = []
    acc = 0

    with tqdm(total=len(data_loader)) as pbar:
        pbar.set_description('validating ')
        for x_batch, y_batch in data_loader:
            x_batch = x_batch.view(x_batch.shape[0], -1)

            y_pred = model(x_batch)

            loss = loss_function(y_pred, y_batch)
            loss_history.append(loss.item())

            for i, pred in enumerate(y_pred):
                if torch.argmax(pred) == y_batch[i]:
                    acc += 1 
            pbar.update(1)
            
    acc /= ((len(data_loader) - 1) * 64 + 32)

    return {"loss": np.mean(loss_history), "accuracy": acc}
    

### Задание 4

Модифицируйте процесс обучения таким образом, чтобы достигнуть наилучшего качества на валидационной выборке. Модель должна оставаться N-слойным перцептроном с количеством обучаемых параметров <= 500000. Для обучения разрешается использовать только набор данных MNIST. Процесс обучения вы можете изменять по собственному усмотрению. К примеру, вы можете менять:

* Архитектуру модели в рамках наложенных ограничений на количество параметров и вид архитектуры (многослойный перцептрон)
* Функции активации в модели
* Используемый оптимизатор
* Расписание шага оптимизации
* Сэмплинг данных при обучении ( e.g. hard negative mining)

В результате мы ожидаем увидеть код экспериментов и любые инсайты, которые вы сможете получить в процессе

### Обучение моделей различных модификаций

#### Обучение предложенной нам модели

In [131]:
model = Perceptron()
print(model)
print('Total number of trainable parameters', 
      sum(p.numel() for p in model.parameters() if p.requires_grad))

Perceptron(
  (structure): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=10, bias=True)
    (7): ReLU()
  )
)
Total number of trainable parameters 335114


In [132]:
train_model(model, 
            train_dataset=train_dataset, 
            val_dataset=val_dataset, 
            loss_function=torch.nn.CrossEntropyLoss(), 
            initial_lr=0.001) # после того, как я поигрался разными параметрами, 
                              # понял, что стоит сделать lr поменьше. Экспериментально
                              # пришел к этому значению. При меньших обучение затягивается,
                              # при больших в начале сильно прыгает loss
"""
Epoch 23
Validation metrics: 
{'loss': 0.04069886377239301, 'accuracy': 0.9880191693290735}
Best model yet, saving
"""

------- Current epoch: 0 -------



Validation metrics: 
{'loss': 0.5761261631728737, 'accuracy': 0.7699680511182109}
Best model yet, saving

------- Current epoch: 1 -------



Validation metrics: 
{'loss': 0.11884576082507677, 'accuracy': 0.9642571884984026}
Best model yet, saving

------- Current epoch: 2 -------



Validation metrics: 
{'loss': 0.08017995446634177, 'accuracy': 0.9746405750798722}
Best model yet, saving

------- Current epoch: 3 -------



Validation metrics: 
{'loss': 0.06726935492869919, 'accuracy': 0.9785343450479234}
Best model yet, saving

------- Current epoch: 4 -------


...


Validation metrics: 
{'loss': 0.05331152061349295, 'accuracy': 0.9841253993610224}

------- Current epoch: 24 -------



Validation metrics: 
{'loss': 0.06371448739968875, 'accuracy': 0.983426517571885}

------- Current epoch: 25 -------



Validation metrics: 
{'loss': 0.06211410421312081, 'accuracy': 0.983326677316294}

------- Current epoch: 26 -------



Validation metrics: 
{'loss': 0.06758038302462752, 'accuracy': 0.9828274760383386}
Early stopping triggered


"\nEpoch 23\nValidation metrics: \n{'loss': 0.04069886377239301, 'accuracy': 0.9880191693290735}\nBest model yet, saving\n"

#### Модели с другими lr_scheduler

In [53]:
train_model(model, 
            train_dataset=train_dataset, 
            val_dataset=val_dataset, 
            loss_function=torch.nn.CrossEntropyLoss(), 
            initial_lr=0.01,
            lr_scheduler_class = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts,
            lr_scheduler_params = {"T_0" : 100})
"""
Данные последней лучшей модели:
    Epoch 26
    Validation metrics: 
    {'loss': 0.11119568567803859, 'accuracy': 0.9728434504792333}
    Best model yet, saving
"""

Epoch 0



Validation metrics: 
{'loss': 0.1741983729924793, 'accuracy': 0.9546725239616614}
Best model yet, saving
Epoch 1



Validation metrics: 
{'loss': 0.1415030888584294, 'accuracy': 0.963158945686901}
Best model yet, saving
Epoch 2


KeyboardInterrupt: 

In [ ]:
train_model(model, 
            train_dataset=train_dataset, 
            val_dataset=val_dataset, 
            loss_function=torch.nn.CrossEntropyLoss(), 
            initial_lr=0.01,
            lr_scheduler_class = torch.optim.lr_scheduler.StepLR,
            lr_scheduler_params = {"step_size" : 50})
"""
Данные последней лучшей модели:
    Epoch 31
    Validation metrics: 
    {'loss': 0.13994677272880343, 'accuracy': 0.9656549520766773}
    Best model yet, saving
"""

"""
Я поменял только способ изменения lr. Loss возрос на 0.02 и не был стабилен.
На протяжении обучения, модель показывала loss вплоть до 0.31(...). Когда модель
с теплым рестартом на lr имела стабильный loss и accuracy у неё по итогу выше.
"""

#### Модели с разными функциями активаций

Посмотрим на модели с той же архитектурой, но другими активациями

In [8]:
model1 = Perceptron(activation = torch.nn.LeakyReLU)
print(model1)
print('Total number of trainable parameters', 
      sum(p.numel() for p in model1.parameters() if p.requires_grad))

Perceptron(
  (layer_1): Linear(in_features=784, out_features=256, bias=True)
  (activation_1): LeakyReLU(negative_slope=0.01)
  (layer_2): Linear(in_features=256, out_features=128, bias=True)
  (activation_2): LeakyReLU(negative_slope=0.01)
  (layer_3): Linear(in_features=128, out_features=64, bias=True)
  (activation_3): LeakyReLU(negative_slope=0.01)
  (layer_4): Linear(in_features=64, out_features=10, bias=True)
)
Total number of trainable parameters 242762


In [9]:
model2 = Perceptron(activation = torch.nn.Softplus)
print(model2)
print('Total number of trainable parameters', 
      sum(p.numel() for p in model2.parameters() if p.requires_grad))

Perceptron(
  (layer_1): Linear(in_features=784, out_features=256, bias=True)
  (activation_1): Softplus(beta=1, threshold=20)
  (layer_2): Linear(in_features=256, out_features=128, bias=True)
  (activation_2): Softplus(beta=1, threshold=20)
  (layer_3): Linear(in_features=128, out_features=64, bias=True)
  (activation_3): Softplus(beta=1, threshold=20)
  (layer_4): Linear(in_features=64, out_features=10, bias=True)
)
Total number of trainable parameters 242762


In [ ]:
"""
Теперь я возьму метод обычения с lr_schedule = CosineAnnealingWarmRestarts
и попробую узнать, какая функция активация лучше. Для тестов я взял LeakyReLU
и Softplus как родственников ReLU. Возможно стоит ещё проверить что-нибудь из 
класса сигмоид
"""
train_model(model1, 
            train_dataset=train_dataset, 
            val_dataset=val_dataset, 
            loss_function=torch.nn.CrossEntropyLoss(), 
            initial_lr=0.01,
            lr_scheduler_class = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts,
            lr_scheduler_params = {"T_0" : 100})
"""
Epoch 7
Validation metrics: 
{'loss': 0.10283183453893392, 'accuracy': 0.9702476038338658}
Best model yet, saving
"""

In [ ]:
train_model(model2, 
            train_dataset=train_dataset, 
            val_dataset=val_dataset, 
            loss_function=torch.nn.CrossEntropyLoss(), 
            initial_lr=0.01,
            lr_scheduler_class = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts,
            lr_scheduler_params = {"T_0" : 100})
"""
Epoch 12
Validation metrics: 
{'loss': 0.12079034244662423, 'accuracy': 0.9681509584664537}
Best model yet, saving
"""

#### Модель с задаваемым числом слоёв и параметров на них
Я считаю это самая прикольная часть задания)

In [134]:
input_resolution = (28, 28)
input_channels = 1
hidden_layer_features = [256, 256, 128, 64, 32]
activation = torch.nn.LeakyReLU
num_classes = 10

BossModel = PerceptronButCooler(input_resolution, input_channels, hidden_layer_features, activation, num_classes)

print(BossModel)
print('Total number of trainable parameters', 
      sum(p.numel() for p in BossModel.parameters() if p.requires_grad))

PerceptronButCooler(
  (structure): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Linear(in_features=64, out_features=32, bias=True)
    (9): LeakyReLU(negative_slope=0.01)
    (10): Linear(in_features=32, out_features=10, bias=True)
    (11): LeakyReLU(negative_slope=0.01)
  )
)
Total number of trainable parameters 310314


In [135]:
train_model(BossModel, 
            train_dataset=train_dataset, 
            val_dataset=val_dataset, 
            loss_function=torch.nn.CrossEntropyLoss(), 
            initial_lr=0.001,
            lr_scheduler_class = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts,
            lr_scheduler_params = {"T_0" : 100})


""""
------- Current epoch: 20 -------
Validation metrics: 
{'loss': 0.050774478572840884, 'accuracy': 0.9848242811501597}
Best model yet, saving
"""

------- Current epoch: 0 -------



Validation metrics: 
{'loss': 0.17956796198620634, 'accuracy': 0.9423921725239617}
Best model yet, saving

------- Current epoch: 1 -------



Validation metrics: 
{'loss': 0.09794328806129919, 'accuracy': 0.9689496805111821}
Best model yet, saving

------- Current epoch: 2 -------



Validation metrics: 
{'loss': 0.09748439573212735, 'accuracy': 0.9686501597444089}
Best model yet, saving

------- Current epoch: 3 -------



Validation metrics: 
{'loss': 0.06885208339307303, 'accuracy': 0.9775359424920128}
Best model yet, saving

------- Current epoch: 4 -------


...


Validation metrics: 
{'loss': 0.06611344242118948, 'accuracy': 0.9842252396166135}

------- Current epoch: 35 -------



Validation metrics: 
{'loss': 0.06384830362209352, 'accuracy': 0.9828274760383386}

------- Current epoch: 36 -------



Validation metrics: 
{'loss': 0.07360716405406974, 'accuracy': 0.983526357827476}

------- Current epoch: 37 -------



Validation metrics: 
{'loss': 0.06597919030174013, 'accuracy': 0.9842252396166135}
Early stopping triggered


'"\n------- Current epoch: 20 -------\nValidation metrics: \n{\'loss\': 0.050774478572840884, \'accuracy\': 0.9848242811501597}\nBest model yet, saving\n'